In [1]:
import pyidi
import os
import dill
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from pixel_setter2 import PixelSetter
from scipy.signal import find_peaks

c:\Users\thijs\anaconda3\lib\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


### Import displacement data

In [2]:
root_disp = 'C:/Users/thijs/Documents/GitHub/pyidi_data/displacements'
file_name_basis = 'Full_web_ecc0_new_Floc4_v1'
file_name_displacemnet = file_name_basis + '_d'
extension = '.pkl'
path = os.path.join(root_disp, file_name_displacemnet + extension)

# Open displacement data
with open(path, 'rb') as f:
    # load the pickle data
    data = pickle.load(f)

# Open corresponding video
file_name_video = file_name_basis + "_S01.cihx"
root_video = 'C:/Users/thijs/Documents/GitHub/pyidi_data/videos'
folder_paths = ['D:/HSC', 'F:/', 'C:/Users/thijs/Documents/HSC/', 'H:/My Drive/PHD/HSC']  #, 'H:/My Drive/PHD/HSC'
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path, topdown=False):
        if file_name_video in files:
            file_path = os.path.join(root, file_name_video)
            print(file_path)
            break
video = pyidi.pyIDI(file_path)
video.info
fs_camera = video.info['Record Rate(fps)']
t_camera = np.arange(video.info['Total Frame']) / fs_camera

H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v1_S01\Full_web_ecc0_new_Floc4_v1_S01.cihx


c:\Users\thijs\anaconda3\lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


### import corresponding impact hammer data

In [3]:
root_impact = 'H:/My Drive/PHD/Data'
files = os.listdir(root_impact)

for file in files:
    if file_name_basis in file[16:-4]:
        path = os.path.join(root_impact, file)
        with open(path, 'rb') as f:
            impact_data = pickle.load(f)
        break
else:
    print(f"{file_name_basis} is not in files")


In [6]:
%matplotlib qt
force_index = impact_data['NI']['channel_names'].index('Force')
accel_index = impact_data['NI']['channel_names'].index('Acceleration')
impact_signal = impact_data['NI']['data'][:,force_index]
accel_signal = impact_data['NI']['data'][:,accel_index]

start_index = np.where(impact_data['NI']['time'] >= 1.5)[0][0]

impact_mean = np.mean(impact_signal[start_index:])
mean_accel = np.mean(accel_signal[start_index:])

impact_signal -= impact_mean
accel_signal = -(accel_signal-mean_accel)/9.81

peaks, _ = find_peaks(impact_data['NI']['data'][:, force_index], height=0.5)


fig, ax = plt.subplots()
plt.plot(impact_data['NI']['time'], impact_signal, label='Force')
plt.plot(impact_data['NI']['time'][peaks], impact_signal[peaks], 'x')
plt.plot(impact_data['NI']['time'], accel_signal, label='Acceleration')
plt.xlabel('Time (s)')
plt.ylabel('Force (N)')
# plt.xlim(impact_data['NI']['time'][peaks[0]]-0.001, impact_data['NI']['time'][peaks[0]]+0.005)


ax2 = ax.twinx()
ax2.plot(t_camera, data[0, :, 0], label='Displacement')
plt.ylabel('pixels (-)')
plt.legend()
plt.show()


In [5]:

data.shape


(2155, 10000, 2)